In [60]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

Requirement already up-to-date: google-api-python-client in /opt/anaconda3/lib/python3.8/site-packages (2.1.0)
Requirement already up-to-date: google-auth-httplib2 in /opt/anaconda3/lib/python3.8/site-packages (0.1.0)
Requirement already up-to-date: google-auth-oauthlib in /opt/anaconda3/lib/python3.8/site-packages (0.4.4)
Note: you may need to restart the kernel to use updated packages.


In [30]:
from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

In [44]:
# import the required libraries
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import os.path
import base64
import re
import email
from bs4 import BeautifulSoup

# Define the SCOPES. If modifying it, delete the token.pickle file.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

# https://stackoverflow.com/questions/619977/regular-expression-patterns-for-tracking-numbers 
matchUPS1      = '/\b(1Z ?[0-9A-Z]{3} ?[0-9A-Z]{3} ?[0-9A-Z]{2} ?[0-9A-Z]{4} ?[0-9A-Z]{3} ?[0-9A-Z]|[\dT]\d\d\d ?\d\d\d\d ?\d\d\d)\b/'
matchUPS2      = '/^[kKJj]{1}[0-9]{10}$/'

matchUSPS0     = '/(\b\d{30}\b)|(\b91\d+\b)|(\b\d{20}\b)/'
matchUSPS1     = '/(\b\d{30}\b)|(\b91\d+\b)|(\b\d{20}\b)|(\b\d{26}\b)| ^E\D{1}\d{9}\D{2}$|^9\d{15,21}$| ^91[0-9]+$| ^[A-Za-z]{2}[0-9]+US$/i'
matchUSPS2     = '/^E\D{1}\d{9}\D{2}$|^9\d{15,21}$/'
matchUSPS3     = '/^91[0-9]+$/'
matchUSPS4     = '/^[A-Za-z]{2}[0-9]+US$/'
matchUSPS5     = '/(\b\d{30}\b)|(\b91\d+\b)|(\b\d{20}\b)|(\b\d{26}\b)| ^E\D{1}\d{9}\D{2}$|^9\d{15,21}$| ^91[0-9]+$| ^[A-Za-z]{2}[0-9]+US$/i'

matchFedex1    = '/(\b96\d{20}\b)|(\b\d{15}\b)|(\b\d{12}\b)/'
matchFedex2    = '/\b((98\d\d\d\d\d?\d\d\d\d|98\d\d) ?\d\d\d\d ?\d\d\d\d( ?\d\d\d)?)\b/'
matchFedex3    = '/^[0-9]{15}$/'

possible_tracking_numbers_regex = [matchUPS1, matchUPS2, matchUSPS0, matchUSPS1, matchUSPS2, matchUSPS3, matchUSPS4, matchUSPS5, matchFedex1, matchFedex2, matchFedex3]

In [65]:
def grab_tracking_number(soup):
    number_set = set()

    usps = '([A-Z]{2}\d{9}[A-Z]{2}|(420\d{9}(9[2345])?)?\d{20}|(420\d{5})?(9[12345])?(\d{24}|\d{20})|82\d{8})'
    ups = '(1Z[A-H,J-N,P,R-Z,0-9]{16})'
    fedex = '\D([0-9]{12}|100\d{31}|\d{15}|\d{18}|96\d{20}|96\d{32})\D'

    pattern = re.compile(f'{usps}|{ups}|{fedex}')
    fedex_pattern = re.compile(fedex)

    for line in open("./test_emails/inked.txt"):
        for match in re.finditer(pattern, line):
            if fedex_pattern.match(match[0]):
                number = re.sub("[^0-9]", "", match[0])
                number_set.add(number)
            else:
                number_set.add(match[0])

    return number_set

In [ ]:
dummy_number

In [79]:
def getTrackingNumbersFromEmailInbox():
    tracking_numbers = set()
    # Variable creds will store the user access token.
    # If no valid token found, we will create one.
    creds = None

    # The file token.pickle contains the user access token.
    # Check if it exists
    if os.path.exists('token.pickle'):

        # Read the token from the file and store it in the variable creds
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    
    
    # If credentials are not available or are invalid, ask the user to log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=8080)

        # Save the access token in token.pickle file for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    # Connect to the Gmail API
    service = build('gmail', 'v1', credentials=creds)

    # request a list of all the messages
    # result = service.users().messages().list(userId='me').execute()
    
    # # We can also pass maxResults to get any number of emails. Like this:
    result = service.users().messages().list(maxResults=10000, userId='me', q ='"tracking number"').execute()
    messages = result.get('messages')

    # messages is a list of dictionaries where each dictionary contains a message id.
    # iterate through all the messages
    for msg in messages:
        # Get the message from its id
        txt = service.users().messages().get(userId='me', id=msg['id']).execute()
        
        # Use try-except to avoid any Errors
        try:
            # print(txt)
            # Get value of 'payload' from dictionary 'txt'
            payload = txt['payload']
            headers = payload['headers']

            # Look for Subject and Sender Email in the headers
            for d in headers:
                if d['name'] == 'Subject':
                    subject = d['value']
                    # tracking_numbers.append(subject)
                if d['name'] == 'From':
                    sender = d['value']

            # The Body of the message is in Encrypted format. So, we have to decode it.
            # Get the data and decode it with base 64 decoder.
            parts = payload.get('parts')[0]
            data = parts['body']['data']
            data = data.replace("-","+").replace("_","/")
            decoded_data = base64.b64decode(data)
            f = open("emaillxml.txt", "x")
            f.write(decoded_data)
            # Now, the data obtained is in lxml. So, we will parse
            # it with BeautifulSoup library
            # soup = BeautifulSoup(decoded_data , "lxml")
            # body = soup.body()

            # TODO: scan through emails and find tracking numbers, add them to tracking_numbers local var
            # Printing the subject, sender's email and message
            tracking_numbers += grab_tracking_number(decoded_data)

            # print("Subject: ", subject)
            # print("From: ", sender)
            # print("Message: ", body)
            # print('\n')
        except:
            pass

    return tracking_numbers

In [80]:
list_of_tracking_numbers = getTrackingNumbersFromEmailInbox()
print(list_of_tracking_numbers)

set()


In [81]:
# TODO: Take list of tracking numbers, send them through the ShipEngine to get all data regarding the tracking number
# TODO: 